In [1]:
import torch
from datasets import Dataset
from transformers import BertTokenizer, BertForMaskedLM, DataCollatorForLanguageModeling, AutoTokenizer, AutoModelForMaskedLM, Trainer, TrainingArguments
import os
import pickle

model_name = 'bert-base-multilingual-cased'
# model_name = 'xlm-roberta-base'

tokenizer = AutoTokenizer.from_pretrained(model_name)

2023-03-26 00:33:35.797928: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-03-26 00:33:36.494997: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2023-03-26 00:33:36.495063: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory
2023-03-26 00:33:36.495071: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Cannot dlopen some TensorRT libraries. If you would like t

# load corpora

In [2]:
# Load your own corpus
def load_custom_corpus(file_path):
    import pandas as pd
    df = pd.read_csv(file_path)
    df = df.fillna('')
    df = df[df['data_type'] == 'unknown'] # only take rows that do not belong to train/dev/test of IFD-EN-5203
    sentences = [sent for sent in df['text']]
    return {'text': sentences}

def tokenize_function(examples):
    return tokenizer((examples['text']), truncation=True, max_length=128, padding='max_length')

file_path_en = '/home/pgajo/working/data/datasets/English/Incels.is/IFC-22-EN_updated.csv'  # Replace this with the path to your corpus file
corpus_en = load_custom_corpus(file_path_en)
dataset_en = Dataset.from_dict(corpus_en)
tokenized_dataset_en = dataset_en.map(tokenize_function, batched=True, remove_columns=['text'])

file_path_it = '/home/pgajo/working/data/datasets/Italian/Il_forum_dei_brutti/IFC-22-IT_updated.csv'  # Replace this with the path to your corpus file
corpus_it = load_custom_corpus(file_path_it)
dataset_it = Dataset.from_dict(corpus_it)
tokenized_dataset_it = dataset_it.map(tokenize_function, batched=True, remove_columns=['text'])
print(tokenized_dataset_en)
print(tokenized_dataset_it)

/tmp/ipykernel_1568959/819416219.py:4: DtypeWarning: Columns (2,7) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file_path)


Map:   0%|          | 0/4752924 [00:00<?, ? examples/s]

Map:   0%|          | 0/627839 [00:00<?, ? examples/s]

Dataset({
    features: ['input_ids', 'token_type_ids', 'attention_mask'],
    num_rows: 4752924
})
Dataset({
    features: ['input_ids', 'token_type_ids', 'attention_mask'],
    num_rows: 627839
})


# create pickles

In [3]:
path_dataset_en = '/home/pgajo/working/data/datasets/English/Incels.is'
filename_pickle_en = 'IFC-22-EN_'+str(int(len(tokenized_dataset_en)/1000))+'k.pickle'
path_pickle_en = os.path.join(path_dataset_en,filename_pickle_en)
print(path_pickle_en)
if not os.path.isfile(path_pickle_en):
    # Save the tokenized_dataset as a pickle file
    with open(path_pickle_en, 'wb') as file:
        pickle.dump(tokenized_dataset_en, file)
else:
    print(f"{path_pickle_en} already exists. Not overwriting.")

path_dataset_it = '/home/pgajo/working/data/datasets/Italian/Il_forum_dei_brutti'
filename_pickle_it = 'IFC-22-IT_'+str(int(len(tokenized_dataset_it)/1000))+'k.pickle'
path_pickle_it = os.path.join(path_dataset_it,filename_pickle_it)
print(path_pickle_it)
if not os.path.isfile(path_pickle_it):
    # Save the tokenized_dataset as a pickle file
    with open(path_pickle_it, 'wb') as file:
        pickle.dump(tokenized_dataset_it, file)
else:
    print(f"{path_pickle_it} already exists. Not overwriting.")

/home/pgajo/working/data/datasets/English/Incels.is/IFC-22-EN_4752k.pickle
/home/pgajo/working/data/datasets/Italian/Il_forum_dei_brutti/IFC-22-IT_627k.pickle


# subsample and create pickles

In [ ]:
# Set a seed to ensure reproducibility when shuffling
seed = 42
sample_n_list = [5_000, 10_000, 50_000, 100_000, 500_000]

for sample_n in sample_n_list:
    shuffled_dataset_en = tokenized_dataset_en.shuffle(seed=seed)
    sampled_dataset_en = shuffled_dataset_en.select(range(sample_n))
    print(sampled_dataset_en)

    shuffled_dataset_it = tokenized_dataset_it.shuffle(seed=seed)
    sampled_dataset_it = shuffled_dataset_it.select(range(sample_n))
    print(sampled_dataset_it)

    path_dataset_en = '/home/pgajo/working/data/datasets/English/Incels.is'
    filename_pickle_en = 'IFC-22-EN_'+str(int(sample_n/1000))+'k.pickle'
    path_pickle_en = os.path.join(path_dataset_en,filename_pickle_en)

    if not os.path.isfile(path_pickle_en):
        # Save the tokenized_dataset as a pickle file
        with open(path_pickle_en, 'wb') as file:
            pickle.dump(sampled_dataset_en, file)
    else:
        print(f"{path_pickle_en} already exists. Not overwriting.")

    path_dataset_it = '/home/pgajo/working/data/datasets/Italian/Il_forum_dei_brutti'
    filename_pickle_it = 'IFC-22-IT_'+str(int(sample_n/1000))+'k.pickle'
    path_pickle_it = os.path.join(path_dataset_it,filename_pickle_it)

    if not os.path.isfile(path_pickle_it):
        # Save the tokenized_dataset as a pickle file
        with open(path_pickle_it, 'wb') as file:
            pickle.dump(sampled_dataset_it, file)
    else:
        print(f"{path_pickle_it} already exists. Not overwriting.")

In [10]:
print(tokenized_dataset_en)
print(tokenized_dataset_it)

Dataset({
    features: ['input_ids', 'token_type_ids', 'attention_mask'],
    num_rows: 4752924
})
Dataset({
    features: ['input_ids', 'token_type_ids', 'attention_mask'],
    num_rows: 637833
})


In [12]:
len(tokenized_dataset_it)

637833

In [13]:
str(int(len(tokenized_dataset_it)/1000))

'637'

In [14]:
path_dataset_en = '/home/pgajo/working/data/datasets/English/Incels.is'
filename_pickle_en = 'IFC-22-EN_'+str(int(len(tokenized_dataset_en)/1000))+'k.pickle'
path_pickle_en = os.path.join(path_dataset_en,filename_pickle_en)

if not os.path.isfile(path_pickle_en):
    # Save the tokenized_dataset as a pickle file
    with open(path_pickle_en, 'wb') as file:
        pickle.dump(tokenized_dataset_en, file)
else:
    print(f"{path_pickle_en} already exists. Not overwriting.")

path_dataset_it = '/home/pgajo/working/data/datasets/Italian/Il_forum_dei_brutti'
filename_pickle_it = 'IFC-22-IT_'+str(int(len(tokenized_dataset_it)/1000))+'k.pickle'
path_pickle_it = os.path.join(path_dataset_it,filename_pickle_it)

if not os.path.isfile(path_pickle_it):
    # Save the tokenized_dataset as a pickle file
    with open(path_pickle_it, 'wb') as file:
        pickle.dump(tokenized_dataset_it, file)
else:
    print(f"{path_pickle_it} already exists. Not overwriting.")

/home/pgajo/working/data/datasets/English/Incels.is/IFC-22-EN_4752k.pickle already exists. Not overwriting.


# load pickles

In [15]:
path_pickle_en = '/home/pgajo/working/data/datasets/English/Incels.is/IFC-22-EN.pickle'

# Load the tokenized_dataset from the pickle file
with open(path_pickle_en, 'rb') as file:
    tokenized_dataset_en = pickle.load(file)

path_pickle_it = '/home/pgajo/working/data/datasets/Italian/Il_forum_dei_brutti/IFC-22-IT.pickle'

# Load the tokenized_dataset from the pickle file
with open(path_pickle_it, 'rb') as file:
    tokenized_dataset_it = pickle.load(file)

FileNotFoundError: [Errno 2] No such file or directory: '/home/pgajo/working/data/datasets/English/Incels.is/IFC-22-EN.pickle'

In [ ]:
print(tokenized_dataset_en)
print(tokenized_dataset_it)

In [ ]:
from datasets import concatenate_datasets

# Assuming you have loaded the two datasets as `dataset1` and `dataset2`
merged_dataset = concatenate_datasets([tokenized_dataset_en, tokenized_dataset_it])
shuffled_dataset = merged_dataset.shuffle(seed=seed)
shuffled_dataset

In [ ]:
# # Load the tokenized_dataset from the pickle file
# with open('/home/pgajo/working/data/datasets/English/Incels.is/IFC-22-EN_empty_datatype.pickle', 'rb') as file:
#     tokenized_dataset_en = pickle.load(file)

In [ ]:
data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=True, mlm_probability=0.15)

model = AutoModelForMaskedLM.from_pretrained(model_name)

training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=1,
    per_device_train_batch_size=32,
    save_steps=10_000,
    save_total_limit=2,
    logging_dir='./logs',
    logging_steps=10,
    report_to='none',
    disable_tqdm = False
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=shuffled_dataset,
    data_collator=data_collator,
    tokenizer=tokenizer,
)

trainer.train(
    # resume_from_checkpoint = True
    )

In [ ]:
from transformers import AutoModelForMaskedLM, AutoTokenizer
import os

# Replace these with the appropriate model and tokenizer names
new_model_name = 'incel-'+model_name+'-'+str(int(len(shuffled_dataset)/1000))+'k'

# Save the model and tokenizer to a directory
output_dir = "/home/pgajo/working/pt_models"

# Create the directory if it doesn't exist
model_path = os.path.join(output_dir,new_model_name)

if not os.path.exists(model_path):
    os.makedirs(model_path)
model_path

In [ ]:
model.save_pretrained(model_path)
tokenizer.save_pretrained(model_path)